### danceability

Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

### energy

Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

### loudness

The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.

### valence

A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

### speechiness

Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

In [ ]:
# get a list of playlists containing different moods for training for personality 


In [37]:
import pandas as pd
import numpy as np
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [149]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = '08923f56c32e46c1ae4b4ff05260b76f'
CLIENT_SECRET = '226fd5b07d874db0a61e720c750d646c'
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [150]:
#pip install spotipy --upgrade -q

In [95]:

#!pip install spotipy --upgrade -q

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
def call_playlist(creator_id, playlist_url, limit, offset):
    # NOTE: use playlist_url instead of playlist_id. playlist_id doesn't accept limit and offset parameters for some reason?
    
    # create an empty list and an empty df
    playlist_features_list = ['artist', 'album', 'track_name', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 
                              'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'acousticness']
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # loop through the specified playlist and extract wanted features
    playlist = sp.user_playlist_tracks(creator_id, playlist_url, limit=limit, offset=offset)["items"]
    for track in playlist:

        # create empty dict
        playlist_features = {}

        # get metadata
        playlist_features['artist'] = track['track']['album']['artists'][0]['name']
        playlist_features['album'] = track['track']['album']['name']
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        # playlist_features["genres"] = sp.artist(track['track']['album']['artists'][0]['id'])['genres'] # extract genres using artist id
        playlist_features["popularity"] = track["track"]["popularity"]
        playlist_features["album_release_date"] = track["track"]["album"]["release_date"]
        playlist_features["duration_ms"] = track["track"]["duration_ms"]
        # playlist_features['added_by'] = track["added_by"]["id"]
        # playlist_features['added_at'] = track["added_at"]
        
        #get genres
        track_info = sp.track(playlist_features["track_id"])
        playlist_features["genres"] = track_info["album"]["genres"]
        
        # get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # concat dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    # return df
    return playlist_df


In [106]:

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
def call_playlist2(creator_id, playlist_url, limit, offset):
    # create an empty list and an empty df
    playlist_features_list = ['artist', 'album', 'track_name', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'genres',
                              'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'acousticness']
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # loop through the specified playlist and extract wanted features
    playlist = sp.user_playlist_tracks(creator_id, playlist_url, limit=limit, offset=offset)["items"]
    for track in playlist:

        # create empty dict
        playlist_features = {}

        # get metadata
        playlist_features['artist'] = track['track']['album']['artists'][0]['name']
        playlist_features['album'] = track['track']['album']['name']
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["popularity"] = track["track"]["popularity"]
        playlist_features["album_release_date"] = track["track"]["album"]["release_date"]
        playlist_features["duration_ms"] = track["track"]["duration_ms"]
        
        # get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:18]:
            playlist_features[feature] = audio_features[feature]
        
        # get genre information
        track_info = sp.track(playlist_features["track_id"])
        playlist_features["genres"] = track_info["album"]["genres"]
        
        # concat dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    # return df
    return playlist_df


In [107]:
username = 'Spotify'
testpls = [
    'https://open.spotify.com/playlist/37i9dQZF1DX7qK8ma5wgG1',
    # 'https://open.spotify.com/playlist/37i9dQZF1DX3YSRoSdA634', #life sucks
    # 'https://open.spotify.com/playlist/37i9dQZF1DWSqBruwoIXkA', #sad hours
    # 'https://open.spotify.com/playlist/37i9dQZF1EIg6gLNLe52Bd' #lonely sad mix
    ]     
test = pd.concat([call_playlist2(username, url, 100, 0) for url in testpls], ignore_index=True)
test['vibe']='test'

KeyError: 'genres'

In [105]:
test["genres"] = sp.track(test["track_id"])["artists"][0]["genres"]

TypeError: expected string or bytes-like object

In [41]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()


In [42]:
username = 'Spotify'

## usually takes 1m44s to load
upbeat_pls = [
    'https://open.spotify.com/playlist/37i9dQZF1DX3rxVfibe1L0', # mood booster
    'https://open.spotify.com/playlist/37i9dQZF1DX66m4icL86Ru', #BBE
    'https://open.spotify.com/playlist/37i9dQZF1DWYBO1MoTDhZI', # good vibes
    'https://open.spotify.com/playlist/37i9dQZF1DWSf2RDTDayIx', # happy beats
    'https://open.spotify.com/playlist/37i9dQZF1DX7KNKjOK0o75', # have a great day
    'https://open.spotify.com/playlist/37i9dQZF1DXdPec7aLTmlC', # happy hits
    'https://open.spotify.com/playlist/37i9dQZF1DX0Uv9tZ47pWo', #girls night
    'https://open.spotify.com/playlist/37i9dQZF1DWXti3N4Wp5xy', #pop party
    'https://open.spotify.com/playlist/37i9dQZF1DWSqmBTGDYngZ', #sing in the shower
    'https://open.spotify.com/playlist/37i9dQZF1DWZixSclZdoFE' #energy boost
]


# call the call_playlist function for each playlist URL and concatenate the results
upbeats = pd.concat([call_playlist(username, url, 100, 0) for url in upbeat_pls], ignore_index=True)
upbeats['vibe']='upbeat'


In [6]:
import os
os.chdir('/Users/lphan/Documents/Spotify')

In [7]:

# upbeats['vibe']='upbeat'
upbeats.to_csv('upbeats.csv',index=False)

In [43]:
#usually takes 
sad_pls = [
    'https://open.spotify.com/playlist/37i9dQZF1DX7qK8ma5wgG1',
    'https://open.spotify.com/playlist/37i9dQZF1DX3YSRoSdA634', #life sucks
    'https://open.spotify.com/playlist/37i9dQZF1DWSqBruwoIXkA', #sad hours
    'https://open.spotify.com/playlist/37i9dQZF1DX64Y3du11rR1',
    'https://open.spotify.com/playlist/37i9dQZF1DX46zHlTTyolY', #heartache
    'https://open.spotify.com/playlist/37i9dQZF1DX3bgBpcV2oGY', #sad guitar instrumental
    'https://open.spotify.com/playlist/37i9dQZF1DX15JKV0q7shD', #classics for crying
    'https://open.spotify.com/playlist/37i9dQZF1DWVV27DiNWxkR', #sad indie
    'https://open.spotify.com/playlist/37i9dQZF1EIfAoIM3ht61G', #mellow mix
    'https://open.spotify.com/playlist/37i9dQZF1EIg6gLNLe52Bd' #lonely sad mix
    ]     
sads = pd.concat([call_playlist(username, url, 100, 0) for url in sad_pls], ignore_index=True)
sads['vibe']='sad'


In [60]:
#chill
#usually takes 
# chill_pls = [
#     'https://open.spotify.com/playlist/37i9dQZF1DX889U0CL85jj', #chill vibes
#     'https://open.spotify.com/playlist/37i9dQZF1DX6VdMW310YC7', #chill tracks
#     'https://open.spotify.com/playlist/37i9dQZF1DX0MLFaUdXnjA', #chill pop
#     'https://open.spotify.com/playlist/37i9dQZF1DWYoYGBbGKurt', #lofi and chill
#     'https://open.spotify.com/playlist/37i9dQZF1DX6tTW0xDxScH', #stay in bed
#     'https://open.spotify.com/playlist/37i9dQZF1DWVtzubo4lvxd', #sunset chill
#     'https://open.spotify.com/playlist/37i9dQZF1DX2yvmlOdMYzV', #lowkey
#     'https://open.spotify.com/playlist/37i9dQZF1DX4sWSpwq3LiO', #peaceful piano
#     'https://open.spotify.com/playlist/37i9dQZF1DWV7EzJMK2FUI', #jazz in the background
#     'https://open.spotify.com/playlist/37i9dQZF1DWWQRwui0ExPn' #lofi beats
#     ]     
chill_pls = ['https://open.spotify.com/playlist/37i9dQZF1DX6VdMW310YC7?si=838257ac8e4941b3',
# 'https://open.spotify.com/playlist/37i9dQZF1DWYoYGBbGKurt', #lofi and chill
# 'https://open.spotify.com/playlist/37i9dQZF1DWVtzubo4lvxd', #sunset chill
'https://open.spotify.com/playlist/37i9dQZF1DX4sWSpwq3LiO', #peaceful piano
    'https://open.spotify.com/playlist/37i9dQZF1DWV7EzJMK2FUI', #jazz in the background
        'https://open.spotify.com/playlist/37i9dQZF1DWWQRwui0ExPn', #lofi beats
'https://open.spotify.com/playlist/37i9dQZF1DX6ziVCJnEm59?si=08d65207a1364aeb', # your fav coffee house
'https://open.spotify.com/playlist/37i9dQZF1DWWxrt1tiKYiX?si=06cb4b6969784b4c', #acoustic pop
'https://open.spotify.com/playlist/37i9dQZF1DX5CdVP4rz81C?si=54ebaad9e5744afb',# fresh and chill
'https://open.spotify.com/playlist/37i9dQZF1DX2yvmlOdMYzV?si=123232cf7bf64774',
'https://open.spotify.com/playlist/37i9dQZF1DXdDIhtzn7T2e?si=f10b69923215453a',
'https://open.spotify.com/playlist/37i9dQZF1DX79Y9Kr2M2tM?si=acbbb208fcd84650',
'https://open.spotify.com/playlist/37i9dQZF1DX889U0CL85jj?si=b594c37639c34a2f',
'https://open.spotify.com/playlist/37i9dQZF1DX0MLFaUdXnjA?si=cdd0bdf300fb493d',
'https://open.spotify.com/playlist/37i9dQZF1DX6tTW0xDxScH?si=0dffd3c341604549']
chills = pd.concat([call_playlist(username, url, 80, 0) for url in chill_pls], ignore_index=True)
chills['vibe']='chill'


In [127]:
chills

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,acousticness,popularity,album_release_date,vibe
0,Yotto,Just Over,Just Over,6vOane9RzzOIITCvbHJ6m2,0.574,0.898,7,-9.510,0,0.0333,0.74,0.1660,0.0768,123.010,174344,4,0.00200,62.0,2023-01-05,chill
1,Tinlicker,Because You Move Me,Because You Move Me,05GvwwTLLID738BbKN1ze0,0.789,0.435,2,-9.506,0,0.1490,0.0418,0.1070,0.1660,122.995,196375,4,0.69100,77.0,2017-07-31,chill
2,Tinlicker,Perfect Mistakes,Voodoo,5cGBQ9W3XtRuRQFgDklDrI,0.558,0.834,5,-8.585,1,0.0366,0.89,0.0987,0.1080,123.998,233805,4,0.00573,64.0,2022-10-04,chill
3,ODESZA,Hopeful,Hopeful,7JXVPV3BIeuDs7JImsuN9P,0.340,0.823,4,-7.181,1,0.0455,0.668,0.1410,0.0389,119.972,246405,4,0.08540,65.0,2023-02-21,chill
4,Sultan + Shepard,"Something, Everything",Salta,5L8R2cyft2HfQ1IlfVAbcz,0.653,0.604,7,-10.177,0,0.0327,0.908,0.2550,0.0768,119.980,222250,4,0.04480,63.0,2021-03-12,chill
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,Joy Crookes,Don't Let Me Down (Demo),Don't Let Me Down - Demo,1VwPTHAfPvcL1ZubK6qabL,0.696,0.140,9,-11.509,1,0.0334,0.000354,0.1480,0.2030,80.831,195535,3,0.81800,54.0,2018-11-29,chill
1026,Lee Fields & The Expressions,My World,Honey Dove,1emxi7e5EWFsJmwSKTuUDH,0.487,0.497,4,-5.776,0,0.0640,0.167,0.1090,0.1950,160.317,246573,4,0.12400,53.0,2009-06-02,chill
1027,Charlotte Day Wilson,ALPHA,Mountains,4JvEicQI7CxOJxuI0xtpim,0.540,0.399,10,-9.127,1,0.0524,0.0239,0.3680,0.1410,117.976,249334,4,0.77000,53.0,2021-07-09,chill
1028,Raveena,Shanti,Love Child,7xlhCrF1qme2kFUmynnaQq,0.907,0.338,7,-10.944,1,0.0897,0.0322,0.0418,0.8480,95.989,163072,4,0.45400,53.0,2017-12-06,chill


In [68]:
moods = pd.concat([upbeats, sads, chills, angers], ignore_index = True)
moods.to_csv("mood_data.csv")

In [115]:
moods.drop_duplicates
moods = moods[~(moods['vibe'] == 'angry')]

In [147]:
test = moods.head(10)
test

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,acousticness,popularity,album_release_date,vibe
0,Miley Cyrus,Flowers,Flowers,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.000005,0.0322,0.646,117.999,200455,4,0.06320,98.0,2023-01-13,upbeat
1,Rema,Calm Down (with Selena Gomez),Calm Down (with Selena Gomez),0WtM2NBVQNNJLh6scP13H8,0.801,0.806,11,-5.206,1,0.0381,0.000669,0.1140,0.802,106.999,239318,4,0.38200,95.0,2022-08-25,upbeat
2,David Guetta,I'm Good (Blue),I'm Good (Blue),4uUG5RXrOk84mYEfFvj3cK,0.561,0.965,7,-3.673,0,0.0343,0.000007,0.3710,0.304,128.040,175238,4,0.00383,94.0,2022-08-26,upbeat
3,Stephen Sanchez,Until I Found You (Em Beihold Version),Until I Found You (with Em Beihold) - Em Beiho...,1Y3LN4zO1Edc2EluIoSPJN,0.551,0.550,10,-5.339,1,0.0286,0.0,0.2020,0.342,101.190,176440,3,0.77700,93.0,2022-04-22,upbeat
4,OneRepublic,I Ain’t Worried (Music From The Motion Picture...,I Ain't Worried,4h9wh7iOZ0GGn8QVp4RAOB,0.704,0.797,0,-5.927,1,0.0475,0.000745,0.0546,0.825,139.994,148486,4,0.08260,93.0,2022-05-13,upbeat
5,Harry Styles,As It Was,As It Was,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,0.0557,0.00101,0.3110,0.662,173.930,167303,4,0.34200,92.0,2022-03-31,upbeat
6,Ruth B.,Safe Haven,Dandelions,2eAvDnpXP5W0cVtiI0PUxV,0.609,0.692,1,-2.958,1,0.0259,0.0,0.0864,0.454,116.959,233720,3,0.01570,90.0,2017-05-05,upbeat
7,Rosa Linn,SNAP,SNAP,76OGwb5RA9h4FxQPT33ekc,0.565,0.636,0,-8.198,1,0.0638,0.00001,0.4470,0.525,170.010,179551,4,0.10700,90.0,2022-03-19,upbeat
8,Meghan Trainor,Takin' It Back,Made You Look,0QHEIqNKsMoOY5urbzN48u,0.838,0.525,10,-3.562,1,0.0665,0.000002,0.0771,0.884,144.981,134256,4,0.34500,90.0,2022-10-21,upbeat
9,Sabrina Carpenter,emails i can't send,Nonsense,6dgUya35uo964z7GZXM07g,0.740,0.697,8,-4.912,1,0.0340,0.0,0.2240,0.732,138.992,163648,4,0.02680,89.0,2022-07-15,upbeat


## get genres for mood playlists

In [153]:

track_uris = test['track_id'].to_list()
dict_genre = {}

for t_uri in track_uris:

    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
   
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
   
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']
   

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [144]:
track_uris =  chills['track_id'].to_list()
dict_genre = {}

In [145]:

for t_uri in track_uris:
 
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
   
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
   
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [146]:
# convert dictionary into dataframe with track_uri as the first column
df_features_genres = pd.DataFrame.from_dict(dict_genre, orient='index')
df_features_genres.insert(0, 'track_uri', df_features_genres.index)
df_features_genres.reset_index(inplace=True, drop=True)

df_features_genres.head()

,track_uri,artist_uri,genres
0,6vOane9RzzOIITCvbHJ6m2,,[]


In [ ]:
mood_genres = moods.merge(df_genre, how = 'left', on ='track_id')
mood_genres_exploded = mood_genres.explode('genres')
mood_genres_exploded.head()

In [78]:
mixed = moods.groupby('track_id')['vibe'].apply(lambda x: list(set(x))).reset_index(name='vibes')

In [79]:
mixed.to_csv('mixed.csv')

In [12]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
import plotly.graph_objs as go

In [22]:
upbeats_feature = upbeats[['danceability', 'energy', 'loudness','speechiness',
                             'valence', 'tempo', 'popularity']]

music_feature_upbeats = pd.DataFrame(scaler.fit_transform(upbeats_feature), columns=upbeats_feature.columns)

In [23]:
sads_feature = sads[['danceability', 'energy', 'loudness','speechiness',
                             'valence', 'tempo', 'popularity']]

music_feature_sads = pd.DataFrame(scaler.fit_transform(sads_feature), columns=sads_feature.columns)

In [28]:
# Create radar chart
fig1 = go.Figure(data=go.Scatterpolar(
    r=list(round(music_feature_upbeats.median(),2)),
    theta=['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Valence','Tempo','Popularity'],
    fill='toself'
))

# Update chart layout
fig1.update_layout(
    polar=dict(
        radialaxis=dict(
            # gridcolor="skyblue",
            visible=True,
            range=[0,1],
            tickfont=dict(color='slategrey')
        )),
    showlegend=False,
    title={'text': "Your Audio Features", 'x': 0.5, 'y': 0.95},
    margin=dict(l=0, r=0, t=60, b=0),
    font=dict(size=14)
)
# sandybrown, seagreen, seashell, sienna, silver,
#             skyblue, slateblue, slategray, slategrey, snow,
#             springgreen, steelblue, tan, teal, thistle, tomato,
#             turquoise, violet, wheat, white, whitesmoke,
#             yellow, yellowgreen
# Show chart
fig1.show()

#change the website color
# from IPython.display import display, HTML
# display(HTML('<style>body{background-color: white;}</style>'))


In [29]:
# Create radar chart
fig2 = go.Figure(data=go.Scatterpolar(
    r=list(round(music_feature_sads.median(),2)),
    theta=['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Valence','Tempo','Popularity'],
    fill='toself'
))

# Update chart layout
fig2.update_layout(
    polar=dict(
        radialaxis=dict(
            # gridcolor="skyblue",
            visible=True,
            range=[0,1],
            tickfont=dict(color='slategrey')
        )),
    showlegend=False,
    title={'text': "Your Audio Features", 'x': 0.5, 'y': 0.95},
    margin=dict(l=0, r=0, t=60, b=0),
    font=dict(size=14)
)
fig2.show()



### Findings:
Upbeat: High energy, high valence, high loudness

Sad: Low energy, low valence, high loudness

In [17]:
my_songs = pd.read_csv('AudioFeaturesTable.csv')

In [30]:
my_features = my_songs[['danceability', 'energy', 'loudness','speechiness',
                             'valence', 'tempo', 'popularity']]

music_feature_me = pd.DataFrame(scaler.fit_transform(my_features), columns=my_features.columns)

In [31]:
# Create radar chart
fig3 = go.Figure(data=go.Scatterpolar(
    r=list(round(music_feature_me.median(),2)),
    theta=['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Valence','Tempo','Popularity'],
    fill='toself'
))

# Update chart layout
fig3.update_layout(
    polar=dict(
        radialaxis=dict(
            # gridcolor="skyblue",
            visible=True,
            range=[0,1],
            tickfont=dict(color='slategrey')
        )),
    showlegend=False,
    title={'text': "Your Audio Features", 'x': 0.5, 'y': 0.95},
    margin=dict(l=0, r=0, t=60, b=0),
    font=dict(size=14)
)
fig3.show()



## c